# In this notebook we explore the influnce of different ways of masking CTCF motis to the prediction score and success time.

There are three ways to go about this (in brackets are corresponding tags);
- just randomly shuffle the whole seq i.e no masking (all random)
- scan for CTCF and randomly shuffle the motifs (random motif masking)
- scan for CTCF and manually shuffle the motifs (manual motif masking)

In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import bioframe 
import pandas as pd
import numpy as np 
import os
import json
from io import StringIO
import seaborn as sns
import random 
import akita_utils 
import pysam
import time 
import h5py
genome_fasta = '/project/fudenber_735/genomes/mm10/mm10.fa'
genome_open = pysam.Fastafile('/project/fudenber_735/genomes/mm10/mm10.fa')

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
import tensorflow as tf
print(tf.__version__)

import pysam
from basenji import dataset, seqnn, dna_io,stream
head_i = 1 #mouse
model_num = 1 #which fold to use

base_dir = '/project/fudenber_735/tensorflow_models/akita/v2/models/'
model_dir = base_dir+"/f"+str(model_num)+"c0/train/"
model_file  = model_dir+'/model'+str(head_i)+'_best.h5'

params_file = model_dir+'/params.json'
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params['model']
    params_train = params['train']
seq_length = params_model['seq_length']
params_model['verbose'] = False
seqnn_model = seqnn.SeqNN(params_model)
print('built')
seqnn_model.restore(model_file, head_i=head_i)
print('restored')

hic_diags = params_model['diagonal_offset']
try:
    target_crop = params_model['trunk'][-2]['cropping']
except:
    target_crop = params_model['target_crop']
target_length_cropped = int((seq_length//2048 - target_crop*2 - hic_diags)* ((seq_length//2048 - target_crop*2 - hic_diags) +1)/2) 
target_map_size = seq_length//2048  - target_crop*2 
triu_tup = np.triu_indices(target_map_size,2)
target_map_size, target_length_cropped, triu_tup[0].shape

2.4.1


2022-07-27 11:42:15.201580: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


built
restored


(512, 130305, (130305,))

# Fetching chromosome data to explore background creation

In [3]:
chromsizes = bioframe.read_chromsizes('/project/fudenber_735/tensorflow_models/akita/v2/data/mm10/sequences.bed')
dframe = pd.DataFrame(chromsizes)
dframe['end'] = dframe['length']+ 1310720
dframe = dframe.reset_index()
dframe.rename(columns = {'index' : 'chrom', 'length':'start'}, inplace = True)
df = bioframe.frac_gc(dframe, bioframe.load_fasta('/project/fudenber_735/genomes/mm10/mm10.fa'), return_input=True)

df

,chrom,start,end,GC
0,chr1,40415232,41725952,0.407777
1,chr1,42708992,44019712,0.424985
2,chr1,42053632,43364352,0.424847
3,chr1,41725952,43036672,0.411407
4,chr1,43036672,44347392,0.417725
...,...,...,...,...
6033,chrX,99241984,100552704,0.424162
6034,chrX,120532992,121843712,0.367963
6035,chrX,139360256,140670976,0.407839
6036,chrX,128737280,130048000,0.370827


In [4]:
# df.iloc[6][['chrom','start','end','GC']]
# plt.hist( df['GC'].values, 100);
# np.percentile(df['GC'].dropna().values, np.linspace(1,99,50))

# Generating a sample for down stream analysis

In [5]:
super_set = []
error = 0.0001

for gc in np.percentile(df['GC'].dropna().values, np.linspace(1,99,200)):
    for i in range(df.shape[0]):
        if gc-error <= df['GC'].values[i] <= gc+error:
            super_set += [i]
            break

super_set = list(set(super_set)); print(f'Whole distribution: {super_set} \n')
sample_set = super_set[:-100]; print(f'Sampled datapoints: {sample_set}')
new_dataframe = df.iloc[[ind for ind in set(sample_set)]] 

Whole distribution: [6, 8, 9, 10, 1034, 523, 13, 15, 1040, 1042, 530, 532, 21, 22, 1049, 27, 2075, 1567, 33, 546, 35, 1060, 37, 41, 42, 43, 555, 45, 1072, 1585, 50, 563, 1077, 53, 55, 54, 1080, 63, 64, 67, 69, 73, 74, 75, 78, 79, 86, 87, 600, 91, 1628, 1117, 94, 93, 96, 1633, 99, 104, 616, 106, 619, 111, 624, 113, 114, 626, 632, 120, 634, 1152, 1153, 128, 129, 136, 140, 142, 1167, 148, 151, 152, 153, 154, 155, 157, 158, 160, 161, 165, 169, 175, 176, 184, 1209, 189, 1216, 1729, 705, 706, 3782, 203, 205, 212, 1238, 215, 218, 219, 220, 737, 738, 743, 232, 752, 245, 759, 248, 762, 763, 252, 260, 261, 773, 267, 272, 273, 786, 274, 789, 278, 279, 1304, 282, 1308, 286, 799, 798, 803, 291, 293, 297, 2862, 303, 816, 306, 308, 311, 825, 1340, 1341, 321, 322, 836, 325, 839, 328, 844, 1871, 1874, 1363, 853, 864, 356, 358, 871, 364, 373, 377, 890, 379, 386, 901, 902, 2950, 392, 399, 402, 918, 1947, 927, 416, 420, 2475, 429, 1454, 434, 1460, 446, 1983, 453, 468, 1493, 991, 997, 998, 488, 493, 2030, 

In [6]:
def mutation_search(seqnn_model,
                    genome_fasta,
                    seq_length,
                    dataframe,
                    max_iters=1,
                    batch_size=6,
                    shuffle_k=8,
                    ctcf_thresh=8,
                    scores_thresh = 5500,
                    scores_pixelwise_thresh = .04,
                    masking=False,
                    timing = False
                    ):
    """
    This function creates flat sequences
    """
    flat_seqs = []
    scores_storage = {}
    num_seqs = dataframe.shape[0]

    mot = ">CCAsyAGrkGGCr\n0.0000\t1.0000\t0.0000\t0.0000\n0.0000\t1.0000\t0.0000\t0.0000\n1.0000\t0.0000\t0.0000\t0.0000\n0.0000\t0.5000\t0.5000\t0.0000\n0.0000\t0.5000\t0.0000\t0.5000\n1.0000\t0.0000\t0.0000\t0.0000\n0.0000\t0.0000\t1.0000\t0.0000\n0.5000\t0.0000\t0.5000\t0.0000\n0.0000\t0.0000\t0.5000\t0.5000\n0.0000\t0.0000\t1.0000\t0.0000\n0.0000\t0.0000\t1.0000\t0.0000\n0.0000\t1.0000\t0.0000\t0.0000\n0.5000\t0.0000\t0.5000\t0.0000"
    motif = pd.read_csv(StringIO(mot), sep="\t", header=0, names=["A", "C", "G", "T"]).values
    motif_window = int(np.ceil(len(motif) / 2))
    mot_shuf = [12, 0, 1, 11, 10, 3, 2, 8, 9, 4, 5, 7, 6, 13]

    for ind in range(num_seqs):
        chrom, start, end, gc = dataframe.iloc[ind][["chrom", "start", "end", "GC"]]
        genome_open = pysam.Fastafile(genome_fasta)
        seq = genome_open.fetch(chrom, start, end).upper()
        seq_1hot = dna_io.dna_1hot(seq)

        t0 = time.time()
        num_iters = 0
        while num_iters < max_iters:
            name = f'{num_iters}'
            scores_set = []
            print("ind",ind,", iter ",num_iters,", k ",shuffle_k,", for", chrom, start, end,)
            seq_1hot_batch = []
            for i in range(batch_size):
                seq_1hot_mut = akita_utils.permute_seq_k(seq_1hot, k=shuffle_k)
                s = akita_utils.scan_motif(seq_1hot_mut, motif)
                if masking == 0:
                    for i in np.where(s > ctcf_thresh)[0]:
                        if len(seq_1hot_mut[i-motif_window:i+motif_window]) == len(mot_shuf):
                            seq_1hot_mut[i-motif_window:i+motif_window] = akita_utils.permute_seq_k(seq_1hot_mut[i-motif_window:i+motif_window], k=2)
                    seq_1hot_batch.append(seq_1hot_mut)
                elif masking == 1:
                    for i in np.where(s > ctcf_thresh)[0]:
                        if len(seq_1hot_mut[i-motif_window:i+motif_window]) == len(mot_shuf):
                            seq_1hot_mut[i-motif_window:i+motif_window] = seq_1hot_mut[i-motif_window:i+motif_window][mot_shuf]

                    seq_1hot_batch.append(seq_1hot_mut)
                else:
                    seq_1hot_batch.append(seq_1hot_mut)

            seq_1hot_batch = np.array(seq_1hot_batch)
            pred = seqnn_model.predict(seq_1hot_batch, batch_size=batch_size)
            scores = np.sum(pred**2, axis=-1).sum(axis=-1)
            scores_pixelwise = np.max(pred**2, axis=-1).max(axis=-1)  
            t1 = time.time()

            if np.any([
                (np.min(scores) < scores_thresh)
                , (np.min(scores_pixelwise) < scores_pixelwise_thresh)]
            ):
                scores_set += [scores]
                best_ind = np.argmin(scores_pixelwise)
                best_seq = seq_1hot_batch[best_ind]
                best_pred = pred[best_ind]
                best_score, best_score_pixelwise = (
                    scores[best_ind],
                    scores_pixelwise[best_ind],
                )
                
                print(
                    "success: best seq, thresh",
                    np.min(scores),
                    " pixelwise",
                    np.min(scores_pixelwise),
                    "time",
                    t1 - t0
                )
                if timing == True:
                    num_iters = max_iters
            else:
                scores_set += [scores]
                best_ind = np.argmin(scores_pixelwise)
                best_seq = seq_1hot_batch[best_ind]
                best_pred = pred[best_ind]
                best_score, best_score_pixelwise = (
                    scores[best_ind],
                    scores_pixelwise[best_ind],
                )
                print(
                "trying: best seq, thresh",
                np.min(scores),
                " pixelwise",
                np.min(scores_pixelwise),
                )
            
            scores_storage[name] = scores_set
            
            num_iters += 1
            if num_iters >= max_iters:
                # scores_set += [scores]
                print(f"max iters exceeded, final time {t1 - t0}")
                if gc :
                    flat_seqs.append([
                    best_seq,
                    best_pred,
                    best_score,
                    best_score_pixelwise,
                    t1 - t0,
                    gc
                ])
                else:
                    flat_seqs.append([
                    best_seq,
                    best_pred,
                    best_score,
                    best_score_pixelwise,
                    t1 - t0
                ])

            

    return scores_storage, flat_seqs

# Focusing on the scores' distributions and time taken to obtain a success trial (with and without masking)

In [ ]:
max_iters=10
batch_size=6
# score = 5500

scores_storage_random_masking, flat_seqs_random_masking = mutation_search(    seqnn_model=seqnn_model,
                                                    genome_fasta=genome_fasta,
                                                    seq_length=seq_length,
                                                    dataframe= new_dataframe,
                                                    max_iters=max_iters,
                                                    batch_size=batch_size,
                                                    masking=0,timing = True)

print('**********************************')

scores_storage_motif_masking, flat_seqs_motif_masking = mutation_search(     seqnn_model=seqnn_model,
                                                    genome_fasta=genome_fasta,
                                                    seq_length=seq_length,
                                                    dataframe= new_dataframe,
                                                    max_iters=max_iters,
                                                    batch_size=batch_size,
                                                    masking=1,timing = True)

print('**********************************')

scores_storage_all_random, flat_seqs_no_mask = mutation_search(        seqnn_model=seqnn_model,
                                                    genome_fasta=genome_fasta,
                                                    seq_length=seq_length,
                                                    dataframe= new_dataframe,
                                                    max_iters=max_iters,
                                                    batch_size=batch_size,timing = True)

ind 0 , iter  0 , k  8 , for chr1 39432192 40742912


2022-07-27 11:43:41.157013: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-27 11:43:41.157542: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz
2022-07-27 11:43:42.358388: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4026531840 exceeds 10% of free system memory.
2022-07-27 11:43:43.074948: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2013265920 exceeds 10% of free system memory.
2022-07-27 11:43:43.750307: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2013265920 exceeds 10% of free system memory.
2022-07-27 11:43:44.506719: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1006632960 exceeds 10% of free system memory.
2022-07-27 11:43:45.185892: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1006632960 exceeds 10% of free system memory.


trying: best seq, thresh 8520.0  pixelwise 0.05295
ind 0 , iter  1 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 8056.0  pixelwise 0.05356
ind 0 , iter  2 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 8640.0  pixelwise 0.0561
ind 0 , iter  3 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 7404.0  pixelwise 0.0475
ind 0 , iter  4 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 8180.0  pixelwise 0.06384
ind 0 , iter  5 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 8664.0  pixelwise 0.04956
ind 0 , iter  6 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 8390.0  pixelwise 0.0626
ind 0 , iter  7 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 7544.0  pixelwise 0.0447
ind 0 , iter  8 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 7444.0  pixelwise 0.04962
ind 0 , iter  9 , k  8 , for chr1 39432192 40742912
trying: best seq, thresh 9430.0  pixelwise 0.058
max iters exceeded, final ti

# Visualizing scores' distribution

In [ ]:
Y_1 = []
Y_2 = []
Y_3 = []

for i in scores_storage_random_masking:
    for j in scores_storage_random_masking[i]:
        Y_1 += j.tolist()

for i in scores_storage_motif_masking:
    for j in scores_storage_motif_masking[i]:
        Y_2 += j.tolist()        

for i in scores_storage_all_random:
    for j in scores_storage_all_random[i]:
        Y_3 += j.tolist()
        
print(scores_storage_motif_masking)

kde_1 = pd.DataFrame(Y_1, columns=["score"])
kde_2 = pd.DataFrame(Y_2, columns=["score"])
kde_3 = pd.DataFrame(Y_3, columns=["score"])
sns.kdeplot(data=kde_1, x="score", bw_adjust=.2, label='random motif masking', fill=True)  
sns.kdeplot(data=kde_2, x="score", bw_adjust=.2, label='manual motif masking', fill=True)    
sns.kdeplot(data=kde_3, x="score", bw_adjust=.2, label='all random', fill=True)    
plt.legend()
plt.show()

# Visualizing time taken to obtain a success trial with masking

In [ ]:
flat_pred_time_random_masking = []
flat_pred_time_motif_masking = []
flat_pred_time_no_mask = []

for i in range(len(sample_set)):
    flat_pred_time_no_mask += [flat_seqs_no_mask[i][4]]
    flat_pred_time_random_masking += [flat_seqs_random_masking[i][4]]
    flat_pred_time_motif_masking += [flat_seqs_motif_masking[i][4]]

kde = pd.DataFrame(flat_pred_time_no_mask, columns=["success time (s)"])    
sns.kdeplot(data=kde, x="success time (s)", bw_adjust=.2, label='all random', fill=True)    

kde = pd.DataFrame(flat_pred_time_motif_masking, columns=["success time (s)"])    
sns.kdeplot(data=kde, x="success time (s)", bw_adjust=.2, label='manual motif mask', fill=True)  

kde = pd.DataFrame(flat_pred_time_random_masking, columns=["success time (s)"])    
sns.kdeplot(data=kde, x="success time (s)", bw_adjust=.2, label='random motif mask', fill=True)  

plt.legend()
plt.show()